# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
data_dir = '../noaa-runtime/data/'

In [2]:
#dst_labs = pd.read_csv(data_dir+'dst_labels.csv' ,index_col=["period", "timedelta"],)
#sat_pos  = pd.read_csv(data_dir+'satellite_positions.csv',index_col=["period", "timedelta"],)
sw       = pd.read_csv(data_dir+'solar_wind.csv',index_col=["period", "timedelta"],)
#sunspots = pd.read_csv(data_dir+'sunspots.csv',index_col=["period", "timedelta"],)

## Interpolate and aggregate other files.

In [3]:
# drop the unnecessary columns
cols_to_drop= ['bx_gsm','by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm', 'source']
sw.drop(cols_to_drop, axis=1, inplace=True)

print ('Droping the columns:', cols_to_drop)
print ('remaining columns:', sw.columns )

Droping the columns: ['bx_gsm', 'by_gsm', 'bz_gsm', 'theta_gsm', 'phi_gsm', 'source']
remaining columns: Index(['bx_gse', 'by_gse', 'bz_gse', 'theta_gse', 'phi_gse', 'bt', 'density',
       'speed', 'temperature'],
      dtype='object')


## Interpolation

In [4]:
print ("The Null values in the data:\n", sw.isnull().sum() )

The Null values in the data:
 bx_gse         325888
by_gse         325888
bz_gse         325888
theta_gse      325888
phi_gse        326388
bt             325888
density        684890
speed          689555
temperature    811768
dtype: int64


In [5]:
sw.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)

In [6]:
print ("shape of the data shape:", sw.shape)
print ("Null values after interpolation:", sw.isnull().sum().sum())
sw.head()

shape of the data shape: (8392320, 9)
Null values after interpolation: 0


bx_gse  by_gse  bz_gse  theta_gse  phi_gse    bt  \
period  timedelta                                                           
train_a 0 days 00:00:00   -5.55    3.00    1.25      11.09   153.37  6.80   
        0 days 00:01:00   -5.58    3.16    1.17      10.10   151.91  6.83   
        0 days 00:02:00   -5.15    3.66    0.85       7.87   146.04  6.77   
        0 days 00:03:00   -5.20    3.68    0.68       6.17   146.17  6.74   
        0 days 00:04:00   -5.12    3.68    0.49       4.62   145.72  6.65   

                         density   speed  temperature  
period  timedelta                                      
train_a 0 days 00:00:00     1.53  383.92     110237.0  
        0 days 00:01:00     1.69  381.79     123825.0  
        0 days 00:02:00     1.97  389.11      82548.0  
        0 days 00:03:00     1.97  389.11      82548.0  
        0 days 00:04:00     1.77  384.26      94269.0

In [7]:
def using_reset_index(df):
    df = df.reset_index(level='period')
    df.index=pd.to_timedelta(df.index)
    return df.groupby('period').resample('H').mean()

In [8]:
sw_ipt_agg_ml=using_reset_index(sw)
print ("shape of the data shape:", sw_ipt_agg_ml.shape)
print ("Null values :", sw_ipt_agg_ml.isnull().sum().sum())
sw_ipt_agg_ml.head(2)

shape of the data shape: (139872, 9)
Null values : 0


bx_gse  by_gse    bz_gse  theta_gse     phi_gse  \
period  timedelta                                                      
train_a 00:00:00  -6.149500  1.6455  0.844667   7.103667  165.420333   
        01:00:00  -6.693833  0.7665  1.265167  10.183333  174.267833   

                         bt   density       speed   temperature  
period  timedelta                                                
train_a 00:00:00   7.140000  1.174167  355.278250  68880.341667  
        01:00:00   7.284833  0.815917  330.080583  52203.225000

## join `dst labels` to the original solar wind data

In [9]:
dst_labs = pd.read_csv(data_dir+'dst_labels.csv' ,index_col=["period", "timedelta"],)
print ("shape of the data dst_labs:", dst_labs.shape)
print ("Null values:", dst_labs.isnull().sum().sum())
dst_labs.head()

shape of the data dst_labs: (139872, 1)
Null values: 0


dst
period  timedelta           
train_a 0 days 00:00:00   -7
        0 days 01:00:00  -10
        0 days 02:00:00  -10
        0 days 03:00:00   -6
        0 days 04:00:00   -2

In [10]:
# sample only hourly data
dst_labs = using_reset_index(dst_labs)

In [11]:
df_sw_dst=sw_ipt_agg_ml.join(dst_labs)
df_sw_dst.head(2)

bx_gse  by_gse    bz_gse  theta_gse     phi_gse  \
period  timedelta                                                      
train_a 00:00:00  -6.149500  1.6455  0.844667   7.103667  165.420333   
        01:00:00  -6.693833  0.7665  1.265167  10.183333  174.267833   

                         bt   density       speed   temperature  dst  
period  timedelta                                                     
train_a 00:00:00   7.140000  1.174167  355.278250  68880.341667   -7  
        01:00:00   7.284833  0.815917  330.080583  52203.225000  -10

In [12]:
print(sw_ipt_agg_ml.shape , dst_labs.shape, df_sw_dst.shape)

(139872, 9) (139872, 1) (139872, 10)


## join `Sunspots data` to the dataframe

In [13]:
sunspots = pd.read_csv(data_dir+'sunspots.csv',index_col=["period", "timedelta"],)
print ("shape of the data sunspots:", sunspots.shape)
print ("Null values:", sunspots.isnull().sum().sum())
sunspots = using_reset_index(sunspots)
sunspots.head()

shape of the data sunspots: (192, 1)
Null values: 0


smoothed_ssn
period  timedelta              
train_a 00:00:00           65.4
        01:00:00            NaN
        02:00:00            NaN
        03:00:00            NaN
        04:00:00            NaN

In [14]:
print(sw_ipt_agg_ml.shape, dst_labs.shape, df_sw_dst.shape, sunspots.shape)

(139872, 9) (139872, 1) (139872, 10) (137643, 1)


the rows are slightly different so, let's join the dataframe anyway and then replace the NULL data by interpolation.

In [15]:
df_sw_dst_ss=df_sw_dst.join(sunspots)
df_sw_dst_ss['smoothed_ssn']=df_sw_dst_ss[['smoothed_ssn']].interpolate(method='linear', limit_direction='forward', axis=0)

In [16]:
print ("shape of all:", sw_ipt_agg_ml.shape, dst_labs.shape, df_sw_dst.shape, sunspots.shape)
print ("Total Null values:", df_sw_dst_ss.isnull().sum().sum())
df_sw_dst_ss.tail(2)

shape of all: (139872, 9) (139872, 1) (139872, 10) (137643, 1)
Total Null values: 0


bx_gse    by_gse    bz_gse  theta_gse     phi_gse  \
period  timedelta                                                               
train_c 2435 days 22:00:00 -2.3850  2.860333 -2.221667 -29.645667  130.601500   
        2435 days 23:00:00 -2.2515  2.619333 -2.821000 -39.064000  130.277667   

                                  bt   density       speed   temperature  dst  \
period  timedelta                                                               
train_c 2435 days 22:00:00  4.716667  2.148917  343.488167  41803.641667  -15   
        2435 days 23:00:00  4.554833  2.283583  343.848750  36191.908333  -14   

                            smoothed_ssn  
period  timedelta                         
train_c 2435 days 22:00:00          42.5  
        2435 days 23:00:00          42.5

## join `Satellite Position` to the dataframe

In [17]:
sat_pos  = pd.read_csv(data_dir+'satellite_positions.csv',index_col=["period", "timedelta"],)


print ("shape of the data sunspots:", sat_pos.shape)
print ("Null values:", sat_pos.isnull().sum().sum())
sat_pos = using_reset_index(sat_pos)
print ("Columns:", sat_pos.columns)
sat_pos.head(2)

shape of the data sunspots: (5828, 6)
Null values: 14382
Columns: Index(['gse_x_ace', 'gse_y_ace', 'gse_z_ace', 'gse_x_dscovr', 'gse_y_dscovr',
       'gse_z_dscovr'],
      dtype='object')


gse_x_ace  gse_y_ace  gse_z_ace  gse_x_dscovr  \
period  timedelta                                                  
train_a 00:00:00   1522376.9   143704.6   149496.7           NaN   
        01:00:00         NaN        NaN        NaN           NaN   

                   gse_y_dscovr  gse_z_dscovr  
period  timedelta                              
train_a 00:00:00            NaN           NaN  
        01:00:00            NaN           NaN

In [18]:
# drop the unnecessary columns
cols_to_drop = ['gse_x_dscovr', 'gse_y_dscovr', 'gse_z_dscovr']
sat_pos.drop(cols_to_drop, axis=1, inplace=True)

print ('Droping the columns:', cols_to_drop)
print ('remaining columns:', sat_pos.columns )
sat_pos.head()

Droping the columns: ['gse_x_dscovr', 'gse_y_dscovr', 'gse_z_dscovr']
remaining columns: Index(['gse_x_ace', 'gse_y_ace', 'gse_z_ace'], dtype='object')


gse_x_ace  gse_y_ace  gse_z_ace
period  timedelta                                 
train_a 00:00:00   1522376.9   143704.6   149496.7
        01:00:00         NaN        NaN        NaN
        02:00:00         NaN        NaN        NaN
        03:00:00         NaN        NaN        NaN
        04:00:00         NaN        NaN        NaN

In [19]:
df_final=df_sw_dst_ss.join(sat_pos)

In [20]:
print ("Total Null values:", df_final.isnull().sum())

Total Null values: bx_gse               0
by_gse               0
bz_gse               0
theta_gse            0
phi_gse              0
bt                   0
density              0
speed                0
temperature          0
dst                  0
smoothed_ssn         0
gse_x_ace       134044
gse_y_ace       134044
gse_z_ace       134044
dtype: int64


Interpolate the above three columns.

In [21]:
cols_to_interp=['gse_x_ace', 'gse_y_ace', 'gse_z_ace']
df_final[cols_to_interp]=df_final[cols_to_interp].interpolate(method='linear', limit_direction='forward', axis=0)
print ('final_shape:', df_final.shape)
print ('Number of nulls:', df_sw_dst_ss.isnull().sum().sum() )
df_final.head(2)

final_shape: (139872, 14)
Number of nulls: 0


bx_gse  by_gse    bz_gse  theta_gse     phi_gse  \
period  timedelta                                                      
train_a 00:00:00  -6.149500  1.6455  0.844667   7.103667  165.420333   
        01:00:00  -6.693833  0.7665  1.265167  10.183333  174.267833   

                         bt   density       speed   temperature  dst  \
period  timedelta                                                      
train_a 00:00:00   7.140000  1.174167  355.278250  68880.341667   -7   
        01:00:00   7.284833  0.815917  330.080583  52203.225000  -10   

                   smoothed_ssn     gse_x_ace      gse_y_ace      gse_z_ace  
period  timedelta                                                            
train_a 00:00:00      65.400000  1.522377e+06  143704.600000  149496.700000  
        01:00:00      65.421154  1.522503e+06  143388.108333  149560.758333

In [22]:
print ("shape of all:", sw_ipt_agg_ml.shape, dst_labs.shape, df_sw_dst.shape, sat_pos.shape, df_final.shape)

shape of all: (139872, 9) (139872, 1) (139872, 10) (139803, 3) (139872, 14)


## Adding the Standard Deviation columns for `by_gse`, `bz_gse`, `bt`, `density`, `speed`

In [23]:
def agg_std(df_final, df_sw):
    cols_to_keep = ['by_gse', 'bz_gse', 'bt', 'density', 'speed']
    df_sw = df_sw[cols_to_keep]
    
    df_sw = df_sw.reset_index(level='period')
    df_sw.index=pd.to_timedelta(df_sw.index)
    df_sw = df_sw.groupby('period').resample('H').std()
    
    # rename the columns for the std
    columns = [c+'_std' for c in df_sw.columns]
    df_sw.columns = columns

    df = df_final.join(df_sw)
    return df

In [24]:
df_final = agg_std(df_final, sw)
print ('df_final.shape:', df_final.shape)
print ('Number of nulls:', df_final.isnull().sum().sum() )
df_final.head(2)

df_final.shape: (139872, 19)
Number of nulls: 0


bx_gse  by_gse    bz_gse  theta_gse     phi_gse  \
period  timedelta                                                      
train_a 00:00:00  -6.149500  1.6455  0.844667   7.103667  165.420333   
        01:00:00  -6.693833  0.7665  1.265167  10.183333  174.267833   

                         bt   density       speed   temperature  dst  \
period  timedelta                                                      
train_a 00:00:00   7.140000  1.174167  355.278250  68880.341667   -7   
        01:00:00   7.284833  0.815917  330.080583  52203.225000  -10   

                   smoothed_ssn     gse_x_ace      gse_y_ace      gse_z_ace  \
period  timedelta                                                             
train_a 00:00:00      65.400000  1.522377e+06  143704.600000  149496.700000   
        01:00:00      65.421154  1.522503e+06  143388.108333  149560.758333   

                   by_gse_std  bz_gse_std    bt_std  density_std  speed_std  
period  timedelta                                                            
train_a 00:00:00     1.224754    0.580028  1.607043     0.479647  15.025364  
        01:00:00     0.907098    0.731278  0.325293     0.698365  17.203299

In [25]:
df_to_save = df_final.reset_index(level='period')
df_to_save.head(2)

,period,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bt,density,speed,temperature,dst,smoothed_ssn,gse_x_ace,gse_y_ace,gse_z_ace,by_gse_std,bz_gse_std,bt_std,density_std,speed_std
timedelta,,,,,,,,,,,,,,,,,,,,
00:00:00,train_a,-6.149500,1.6455,0.844667,7.103667,165.420333,7.140000,1.174167,355.278250,68880.341667,-7,65.400000,1.522377e+06,143704.600000,149496.700000,1.224754,0.580028,1.607043,0.479647,15.025364
01:00:00,train_a,-6.693833,0.7665,1.265167,10.183333,174.267833,7.284833,0.815917,330.080583,52203.225000,-10,65.421154,1.522503e+06,143388.108333,149560.758333,0.907098,0.731278,0.325293,0.698365,17.203299


In [26]:
df_to_save.tail(2)

,period,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bt,density,speed,temperature,dst,smoothed_ssn,gse_x_ace,gse_y_ace,gse_z_ace,by_gse_std,bz_gse_std,bt_std,density_std,speed_std
timedelta,,,,,,,,,,,,,,,,,,,,
2435 days 22:00:00,train_c,-2.3850,2.860333,-2.221667,-29.645667,130.601500,4.716667,2.148917,343.488167,41803.641667,-15,42.5,1426937.9,234813.7,140701.8,0.876128,1.328527,0.202373,0.680983,5.550519
2435 days 23:00:00,train_c,-2.2515,2.619333,-2.821000,-39.064000,130.277667,4.554833,2.283583,343.848750,36191.908333,-14,42.5,1426937.9,234813.7,140701.8,0.488619,0.599527,0.049076,0.918391,2.906941


In [27]:
df_to_save.to_csv('processed_df.csv')